# Let's use artificial dataset

Recently I made some artificial data. I simply did copy the disease bbox area and paste to normal image ( which was originally tagged 14, normal class ). and made new annotation for pasted bbox coordinates. You can easily find my dataset [link](https://www.kaggle.com/seokhyunseo/artificial-vinbigdatacoco-format). Please upvote :). I will upload the dataset making codes soon, after I did some refactoring.

# Result  

The baseline result is from [here](https://www.kaggle.com/seokhyunseo/let-s-use-mmdetections-different-models)(not closed notebook, I will upload more contents there and don't forget upvote). I use same validation set for checking

| model | 5epoch | 10epoch | 
|---:|---:|---:|
|CascadeRCNN(Resnet50)|0.182|0.239|
|CascadeRCNN(New dataset) | 0.201 | 0.269 |

I found the more dataset leads to more performance. The quality of artificial images will lead more performance.


In [ ]:
!pip -qq install mmcv-full

In [ ]:
!git clone https://github.com/open-mmlab/mmdetection.git
    
%cd mmdetection

!pip -qq install -e .

In [ ]:
# Pretrain Model Download

!mkdir checkpoints
!wget -c http://download.openmmlab.com/mmdetection/v2.0/cascade_rcnn/cascade_rcnn_r50_caffe_fpn_1x_coco/cascade_rcnn_r50_caffe_fpn_1x_coco_bbox_mAP-0.404_20200504_174853-b857be87.pth -O checkpoints/cascade_rcnn_r50_caffe_fpn_1x_coco_bbox_mAP-0.404_20200504_174853-b857be87.pth

In [ ]:
from mmcv import Config
from mmdet.apis import set_random_seed
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector, init_detector, inference_detector

from IPython.display import clear_output

In [ ]:
## Configuration Setting
cfg = Config.fromfile('./configs/cascade_rcnn/cascade_rcnn_r50_fpn_1x_coco.py')
DATASET_TYPE = 'CocoDataset'
PREFIX = '../../input/artificial-vinbigdatacoco-format/'
cfg.dataset_type = DATASET_TYPE
cfg.classes = ("Aortic_enlargement", "Atelectasis", 
               "Calcification", "Cardiomegaly", 
               "Consolidation", "ILD", "Infiltration", 
               "Lung_Opacity", "Nodule/Mass", "Other_lesion", 
               "Pleural_effusion", "Pleural_thickening", 
               "Pneumothorax", "Pulmonary_fibrosis")

In [ ]:
cfg.data.train.img_prefix = PREFIX
cfg.data.train.classes = cfg.classes
cfg.data.train.ann_file = PREFIX + 'train_annotations.json'
cfg.data.train.type = DATASET_TYPE


cfg.data.val.img_prefix = PREFIX
cfg.data.val.classes = cfg.classes
cfg.data.val.ann_file = PREFIX + 'val_annotations.json'
cfg.data.val.type = DATASET_TYPE



cfg.data.test.img_prefix = PREFIX
cfg.data.test.classes = cfg.classes
cfg.data.test.ann_file = PREFIX + 'val_annotations.json'
cfg.data.test.type = DATASET_TYPE

In [ ]:
for i in cfg.model.roi_head.bbox_head:
    i.num_classes = 14
    
# for i in cfg.model.roi_head.mask_head:
#     i.num_classes = 14

In [ ]:
cfg.optimizer.lr = 0.02 / 8
cfg.lr_config.warmup = None
cfg.log_config.interval = 100

# Change the evaluation metric since we use customized dataset.
cfg.evaluation.metric = 'bbox'
# We can set the evaluation interval to reduce the evaluation times
cfg.evaluation.interval = 5
# We can set the checkpoint saving interval to reduce the storage cost
cfg.checkpoint_config.interval = 5

# Set seed thus the results are more reproducible
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)

# we can use here mask_rcnn.
cfg.load_from = './checkpoints/cascade_rcnn_r50_caffe_fpn_1x_coco_bbox_mAP-0.404_20200504_174853-b857be87.pth'
cfg.work_dir = "../vinbig_output"

cfg.runner.max_epochs = 10
cfg.total_epochs = 10

In [ ]:
clear_output()
model = build_detector(cfg.model)
datasets = [build_dataset(cfg.data.train)]

In [ ]:
train_detector(model, datasets[0], cfg, distributed=False, validate=True)

In [ ]:
import os
os.chdir('../')

In [ ]:
!python ./mmdetection/tools/analysis_tools/analyze_logs.py plot_curve ./vinbig_output/None.log.json --keys s2.loss_cls --legend s2.loss_cls --out "loss_cls.jpg"
!rm -rf "./mmdetection"